# Implementación de un modelo de Machine Learning para predecir enfermedades cardíacas

## Clasificación binaria de datos tabulares usando XGBoost


### Creando la función predict_heart_disease

El conjunto de datos seleccionado para entrenar el modelo es "Heart Disease Dataset", el cual considera las siguientes variables o atributos:<br>
age: edad del paciente.<br>
sex: sexo (1 = hombre, 0 = mujer).<br>
cp: tipo de dolor en el pecho (categorías de 1 a 4, denominadas 1: angina típica, 2: angina atípica, 3: dolor torácico que no es causado por angina, 4: asintomático). <br>
trestbps: presión arterial en reposo (en mm Hg). <br>
chol: nivel de colesterol en sangre (mg/dl). <br>
fbs: glucemia en ayunas (> 120 mg/dl, 1 = verdadero; 0 = falso). <br>
restecg: resultados del electrocardiograma en reposo (0: normal, 1: tiene anomalías de la onda ST-T (inversiones de la onda T y/o elevación o depresión ST de > 0.05 mV), 2: muestra probablemente o definitivamente hipertrofia del ventrículo izquierdo según los criterios de Estes).<br>
thalach: frecuencia cardíaca máxima alcanzada.<br>
exang: angina inducida por ejercicio (1 = sí, 0 = no).<br>
oldpeak: depresión ST inducida por el ejercicio en relación al reposo.<br>
slope: Pendiente del segmento ST del ejercicio máximo. (1: ascendente,
2: plana, 3: descendente)<br>
ca: número de vasos principales coloreados por fluoroscopia (de 0 a 3).<br>
thal: talasemia (1 = normal, 2 = defecto fijo, 3 = defecto reversible).<br>
target: variable objetivo (1 = presencia de enfermedad, 0 = ausencia).

In [2]:
import joblib
import numpy as np

xgb = joblib.load("./model/heart_model.joblib")

def predict_heart_disease(features_patient, confidence):
    """Recibe un vector de características de un paciente y predice si padece o no una enfermedad cardíaca.

    Argumentos:
        features_trip (array): Características del viaje, vector de tamaño 14.
        confidence (float, opcional): Nivel de confianza. Por defecto es 0.5.
    """

    pred_value = xgb.predict_proba(features_patient.reshape(1, -1))[0][1]
    if pred_value >= confidence:
      return 1
    else:
      return 0

Probar con un paciente de ejemplo:

In [5]:
features_patient = np.array([71,0,0,112,149,0,1,125,0,1.6,1,0,2])

In [6]:
predict_heart_disease(features_patient, 0.5)

1

In [7]:
predict_heart_disease(features_patient, 0.6)

1

In [8]:
predict_heart_disease(features_patient, 0.7)

1

Implementando el modelo XGBoost usando fastAPI

In [9]:
import io
import uvicorn
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from pydantic import BaseModel

In [10]:
# Asignamos una instancia de la clase FastAPI a la variable "app".
# Interacturaremos con la API usando este elemento.
app = FastAPI(title='Implementando un modelo de predicción de presencia de enfermedades cardíacas usando FastAPI')

# Creamos una clase para el vector de features de entrada
class Item(BaseModel):
    age: int
    sex: int
    cp:  int
    trestbps: int
    chol: int
    fbs:  int
    restecg: int
    thalach: int
    exang: int
    oldpeak: float
    slope: int
    ca: int
    thal: int

# Usando @app.get("/") definimos un método GET para el endpoint / (que sería como el "home").
@app.get("/")
def home():
    return "¡Felicitaciones! Tu API está funcionando según lo esperado. Anda ahora a http://localhost:8000/docs."


# Este endpoint maneja la lógica necesaria para clasificar.
# Requiere como entrada el vector de características del viaje y el umbral de confianza para la clasificación.
@app.post("/predict")
def prediction(item: Item, confidence: float):


    # 1. Correr el modelo de clasificación
    features_patient = np.array([item.age, item.sex, item.cp, item.trestbps, item. chol, item.fbs,
                    item.restecg, item.thalach, item.exang, item.oldpeak, item.slope, item.ca, item.thal])
    pred = predict_heart_disease(features_patient, confidence)

    # 2. Transmitir la respuesta de vuelta al cliente

    # Retornar el resultado de la predicción
    return {'predicted_class': pred}

In [11]:
# Esto deja correr al servidor en un ambiente interactivo como un Jupyter notebook
nest_asyncio.apply()

# Donde se hospedará el servidor
host = "127.0.0.1"

# ¡Iniciemos el servidor!
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [390]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [390]
